<a href="https://colab.research.google.com/github/KacperGrodecki/nieruchomosci-mazowieckie/blob/0.0.4/Kopia_notatnika_mazowieckie_colab_DNN_mieszkania_pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [163]:
from sklearn.metrics import confusion_matrix
from sklearn import svm
import pandas as pd
import numpy as np
np.random.seed(7)
from google.colab import drive
import seaborn as sns
drive.mount('/content/drive')
import matplotlib.pyplot as plt
import pandas as pd
#from otoDomScraper import daneDomu
#from random import randrange
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt
import seaborn as sns
import re
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
import statistics
from sklearn.cluster import AffinityPropagation
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.decomposition import FastICA
from sklearn import preprocessing
from IPython.display import Javascript
import requests
from collections import OrderedDict
import seaborn as sns
import os
import ipywidgets as widgets

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [164]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [165]:
!wget -q https://dlcdn.apache.org/spark/spark-3.1.2/spark-3.1.2-bin-hadoop3.2.tgz

In [166]:
!tar xf spark-3.1.2-bin-hadoop3.2.tgz

In [167]:
!pip install -q findspark

In [168]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop3.2"

In [169]:
import findspark
findspark.init()

In [170]:
findspark.find()

'/content/spark-3.1.2-bin-hadoop3.2'

In [171]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [172]:
spark

In [173]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')

--2021-10-08 12:20:31--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.161.241.46, 52.202.168.65, 54.237.133.81, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.161.241.46|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.2’

ngrok-stable-linux- 100%[===================>]  13.19M  6.63MB/s    in 2.0s    

2021-10-08 12:20:34 (6.63 MB/s) - ‘ngrok-stable-linux-amd64.zip.2’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [174]:
!curl -s http://localhost:4040/api/tunnels

{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://223a-34-134-215-57.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}},{"name":"command_line (http)","uri":"/api/tunnels/command_line%20%28http%29","public_url":"http://223a-34-134-215-57.ngrok.io","proto":"http","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0},"http":{"count":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95":0,"p99":0}}}],"uri":"/api/tunnels"}


In [175]:
from pyspark.sql.functions import udf

In [176]:
def locCities(cityData):
  locationCities=[]
  for data in cityData:
    locationCities.append(data)
  locationCities=pd.DataFrame(np.array(locationCities),columns=['cityX','cityY','cityGeo'])
  return locationCities

def locDistricts(districtsData):
  locationCities=[]
  for data in cityData:
    locationDistricts.append(data)
  locationDistricts=pd.DataFrame(np.array(locationDistricts),columns=['cityX','cityY','cityGeo'])
  return locationDistricts

def locCounties(countyData):
  locationCounty=[]
  for data in countyData:
    locationCounty.append(data)
  locationCounty=pd.DataFrame(np.array(locationCounty),columns=['countyX','countyY','countyGeo'])
  return locationCounty

In [177]:
def selectLocationX(x):
 # print(x[17],x[20],x[23])
  if x[2]<min(x[5],x[8]):
    #print('min 17',x[15],x[16])
    return x[0]
  elif x[5]<min(x[2],x[8]):
    #print('min 20 ',x[18],x[19])
    return x[3]
  elif x[8]<min(x[2],x[5]):
    #print('min 23 ',x[21],x[22])
    return x[6]

def selectLocationY(x):
 # print(x[17],x[20],x[23])
  if x[2]<min(x[5],x[8]):
    #print('min 17',x[15],x[16])
    return x[1]
  elif x[5]<min(x[2],x[8]):
    #print('min 20 ',x[18],x[19])
    return x[4]
  elif x[8]<min(x[2],x[5]):
    #print('min 23 ',x[21],x[22])
    return x[7]

In [178]:
from pyspark.sql.types import StructType, StructField, IntegerType,StringType,FloatType

In [179]:
def toNum2(txt):
    if type(txt) is int:
        return txt
    elif (type(txt) is str):
        digs = re.findall(r'\d+', txt)
        if len(digs) == 1:
            return int(digs[0])
        elif len(digs) == 2:
            return 1000 * int(digs[0]) + int(digs[1])
        elif len(digs) == 3:
            return 1000000 * int(digs[0]) + 1000 * int(digs[1]) + int(digs[0])

    #   return int(digs)

udf_toNum2 = udf(toNum2, IntegerType())


def toNum1(txt):
    print(txt)
    if type(txt) is str:
        digs = re.findall(r'\d+', txt)
        if len(digs) == 1:
            return float(digs[0])
        elif len(digs) == 2 and (txt[1] != ' '):
            return float(digs[0]) + 0.01 *float(digs[1])
        elif len(digs) == 3:
            return 1000 * float(digs[0]) + float(digs[1]) + 0.001 * float(digs[2])
        elif (type(txt) is str) and (txt[1] == ' '):
            digs = re.findall(r'\d+', txt)
            return 1000 * float(digs[0]) + float(digs[1])
    else:
        return txt

udf_toNum1 = udf(toNum1, FloatType())


def toNum3(txt):
    if type(txt) == int:
        return txt
    elif type(txt) == float:
      if np.isnan(txt):
        return 0
      else:
        return int(txt)
    elif type(txt) == str:
      return float(re.findall(r'\d+', txt)[0])
    else:
      return float(0)

udf_toNum3 = udf(toNum3, FloatType())

def pietra(txt):
    if type(txt) is str:
        if '0' in txt:
            return 0
        if '1' in txt:
            return 1
        elif '2' in txt:
            return 2
        elif '3' in txt:
            return 3
        elif 'parterowy' in txt:
            return 0
    elif type(txt)==float:
      if np.isnan(txt):
        return 0
      else:
        return int(txt)
    elif type(txt)==int:
      return txt
    else:
      return txt

udf_pietra = udf(pietra, IntegerType())      

def cities(x):
    dist=x.split()[4]
    #city=x.split()[5]
    if dist=='warszawski':
        return x.split()[6].lower()
    elif dist in ['Warszawa','Radom','Płock','Siedlce']:
        return dist
    else:
        try:
            return x.split()[5].lower()
        except:
            return 'unknown'

udf_cities = udf(cities, StringType())      


def region(x):
    if x.split()[4]=='Warszawa':
        try:
            return x.split()[5].lower()
        except:
            return ''
    else:
        return ''

udf_region = udf(region, StringType())    

In [180]:
from pyspark.sql.functions import split
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import when

In [181]:
dfCities=spark.read.csv('/content/drive/MyDrive/domymazowieckie/dfcities.csv')
dfCities1=pd.read_csv('/content/drive/MyDrive/domymazowieckie/dfcities.csv')

dfCounties=spark.read.csv('/content/drive/MyDrive/domymazowieckie/dfCounties.csv')
dfCounties1=pd.read_csv('/content/drive/MyDrive/domymazowieckie/dfCounties.csv')

dfVoivodeships=spark.read.csv('/content/drive/MyDrive/domymazowieckie/dfVoivodeships.csv')

dfWarsawDistricts=spark.read.csv('/content/drive/MyDrive/domymazowieckie/dfWarsawDistricts.csv')
dfWarsawDistricts1=pd.read_csv('/content/drive/MyDrive/domymazowieckie/dfWarsawDistricts.csv')

In [182]:
def funCities(x):
  print('function')
  city=x
  rows=dfCities1.loc[dfCities1['city']==city]
  precision=0
  print(x)
  if rows.shape[0] == 0:
    print('row 0')
    return float(0) #0,0,100
  elif rows.shape[0] == 1:
    i=0
    print('row 1')
    longitude=rows.iloc[0,2]#rows.collect()[0][2]
    latitude=rows.iloc[0,1]#rows.collect()[0][1]
    precision=rows.iloc[0,3]#rows.collect()[0][3]
    return float(longitude) #longitude[0],latitude[0],precision[0]
  elif rows.shape[0] > 1:
    print('row >1')
    prec1,prec2=rows.iloc[0,3] ,rows.iloc[1,3] #rows.collect()[0][3],rows.collect()[1][3]
    print('funCities',prec1,prec2)
    if prec1<prec2:
      precision=prec1
      longitude=rows.iloc[0,2]
      latitude=rows.iloc[0,1]
    else:
      precision=prec2
      longitude=rows.iloc[1,2]
      latitude=rows.iloc[1,1]
    return float(longitude) #longitude,latitude,precision
  else:
    return float(0) 

udf_funCities = udf(funCities, FloatType())  

def funCities1(x):
  print('function')
  rows=dfCities[dfCities['_c0']==x.lower()]
  precision=0
  print(x)
  if rows.count() == 0:
    print('row 0')
    return 0 #0,0,100
  elif rows.count() == 1:
    print('row 1')
    longitude=rows.collect()[0][2]
    latitude=rows.collect()[0][1]
    precision=rows.collect()[0][3]
    return longitude[0] #longitude[0],latitude[0],precision[0]
  elif rows.count() > 1:
    print('row >1')
    prec1,prec2=rows.collect()[0][3],rows.collect()[1][3]#rows.iloc[0,3],rows.iloc[1,3]
    if prec1<prec2:
      precision=prec1
      longitude=rows.collect()[0][2]
      latitude=rows.collect()[0][1]
    else:
      precision=prec2
      longitude=rows.collect()[1][2]
      latitude=rows.collect()[1][1]
    return longitude #longitude,latitude,precision
  else:
    return 0 
  
def funDistricts(x):
  rows=dfWarsawDistricts1[dfWarsawDistricts1['warsaw_district']==x]
  #print(x, ' ',rows.shape[0])
  if rows.shape[0]==0:
    return 0,0,100
  elif rows.shape[0]==1:
    longitude=rows.longitude.values
    latitude=rows.latitude.values
    precision=rows.geoPrecision.values
    return longitude[0],latitude[0],precision[0]

udf_funDistricts = udf(funDistricts, FloatType()) 


def funCounties(x):
  try:
    rows=dfCounties1['powiat '+x== dfCounties1['county']]
  except Exception as e:
    return e
  #print(x, ' ',rows.shape[0])
  if rows.shape[0]==0:
    return 0,0,100
  elif rows.shape[0]==1:
    longitude=rows.longitude.values
    latitude=rows.latitude.values
    precision=rows.geoPrecision.values
    return longitude[0],latitude[0],precision[0]
udf_funCounties = udf(funCounties, FloatType())   

In [183]:
dfCities1.head()

,Unnamed: 0,city,latitude,longitude,geoPrecision,wikidata_item_id
0,0,aleksandrów łódzki,51.816667,19.300000,0.016667,Q664371
1,1,alwernia,50.060560,19.539530,0.000010,Q983896
2,2,alwernia,50.066667,19.516667,0.016667,Q983896
3,3,andrychów,49.854970,19.338340,0.139333,Q984857
4,4,annopol,50.883333,21.866667,0.016667,Q567332


In [190]:
def makeDataFrame(file):

    schema = StructType([
    StructField("id", IntegerType(), True),                
    StructField("dzielnica", StringType(), True),
    StructField("powierzchnia", StringType(), True),
    StructField("lPokoi",FloatType(), True),
    StructField("powierzchniaDzialki", FloatType(), True),
    StructField("rodzajZabudowy", StringType(), True),
    StructField("materialBudynku", StringType(), True),
    StructField("rokBudowy", FloatType(), True),
    StructField("stanWykonczenia", StringType(), True),
    StructField("okna", StringType(), True),
    StructField("rynek", StringType(), True),
    StructField("lPieter", FloatType(), True),
    StructField("cena", StringType(), True),])
    concat=spark.read.csv(file,schema=schema)
    #########
    print(type(concat))
    #concat['powierzchnia_corr'] = concat['powierzchnia'].apply(lambda x: toNum1(x))
    #df = df.withColumn("message", udf_myFunction("_3"))
    concat= concat.withColumn('powierzchnia_corr',udf_toNum1('powierzchnia'))
    print('powierzchnia_corr')
    #concat.show()
    #####
    #concat['powierzchniaDzialki_corr'] = concat['powierzchniaDzialki'].apply(lambda x: toNum2(x))
    concat= concat.withColumn('powierzchniaDzialki_corr',udf_toNum2('powierzchniaDzialki'))
    print('powierzchniaDzialki_corr')
    #concat.show()
    #concat['cena_corr'] = concat['cena'].apply(lambda x: toNum2(x))
    concat= concat.withColumn('cena_corr',udf_toNum2('cena'))
    print('cena_corr')
    #concat.show()
    #concat['rokBudowy_corr'] = concat['rokBudowy'].apply(lambda x: toNum3(x))
    concat= concat.withColumn('rokBudowy_corr',udf_toNum3('rokBudowy'))
    print('rokBudowy_corr')
    #concat.show()
    #concat['cena/m'] = concat['cena_corr'] / concat['powierzchnia_corr']####??
    concat= concat.withColumn('cena/m',concat['cena_corr']/concat['powierzchnia_corr'])
    print('cena/m')
    #concat.show()
    #concat['lPieter_crr'] = concat['lPieter'].apply(lambda x: pietra(x))
    concat= concat.withColumn('lPieter_crr',udf_pietra('lPieter'))
    print('lPieter_crr')
   # concat.show()
    concat = concat[concat['powierzchnia_corr'] > 0]
    concat = concat[concat['cena_corr'] > 0]
    print('filtr powierzchnia i cena')
   # concat.show()
    #concat['lPokoi'] = concat['lPokoi'].apply(lambda x: toNum3(x))
    concat= concat.withColumn('lPokoi',udf_toNum3('lPokoi'))
  #  concat.show()
    #concat['districts']=concat['dzielnica'].apply(lambda x: x.split()[4])
    ##https://stackoverflow.com/questions/39235704/split-spark-dataframe-string-column-into-multiple-columns
    dzielnica = split(concat['dzielnica'], ' ')
    concat = concat.withColumn('districts', dzielnica.getItem(4))
    print('districts')
   # concat.show()    
    concat= concat.withColumn('cities_corr',udf_cities('dzielnica'))
    print('cities_corr')
   # concat.show()        
    #concat['cities_corr']=cities_corr

    #region_corr=concat['dzielnica'].apply(lambda x: region(x))
    concat= concat.withColumn('region_corr',udf_region('dzielnica'))
   # concat['region_corr']=region_corr
    print('region_corr')
   # concat.show()  

    concat_dropped = concat.drop('dzielnica', 'powierzchnia', 'powierzchniaDzialki', 'lPieter', 'cena', 'cena_corr')
    print('concat_dropped')
    concat_dropped.show()  
    
    #data_df = data_df.withColumn("Plays", data_df["Plays"].cast(IntegerType()))
    
    #concat_dropped['rokBudowy_corr'] = concat_dropped['rokBudowy_corr'].astype('int')
    concat_dropped= concat.withColumn('rokBudowy_corr',concat['rokBudowy_corr'].cast(IntegerType()))
    print('rokBudowy_corr')
    #concat_dropped.show()  
    concat_dropped = concat_dropped.withColumn("rokBudowy_corr", when(concat_dropped["rokBudowy_corr"] < 0, 1900).otherwise(concat_dropped["rokBudowy_corr"]))
    #concat_dropped[concat_dropped['rokBudowy_corr']<1900,'rokBudowy_corr']=1980
    print('korekta na rok')
    #concat_dropped.show()  
    #concat_dropped[concat_dropped['rokBudowy_corr']>2030,'rokBudowy_corr']=2020
    concat_dropped = concat_dropped.withColumn("rokBudowy_corr", when(concat_dropped["rokBudowy_corr"] > 2030, 2020).otherwise(concat_dropped["rokBudowy_corr"]))
    #concat_dropped = concat_dropped[concat_dropped['rokBudowy_corr'] < 2030]
    concat_dropped = concat_dropped.fillna(0)
    print('fill nan')
    #concat_dropped.show()  
    concat_dropped = concat_dropped[concat_dropped['cena/m'] < 20000]
    print('korekta na cene')
    #concat_dropped.show()  
    
    #cityData=concat_dropped.cities_corr.apply(lambda x: funCities(x))
    concat_dropped= concat_dropped.withColumn("locationCities",udf_funCities("cities_corr"))
    #https://stackoverflow.com/questions/38984775/spark-errorexpected-zero-arguments-for-construction-of-classdict-for-numpy-cor
    #locationCities=locCities(cityData)
    print('locationCities')
    concat_dropped.show()
    #districtsData=concat_dropped.region_corr.apply(lambda x: funDistricts(x))
    concat_dropped= concat_dropped.withColumn("locationDistricts",udf_funDistricts("region_corr"))
    #locationDistricts=locCities(districtsData)
    print('locationDistricts')
    concat_dropped.show()
    #countyData=concat_dropped.districts.apply(lambda x: funCounties(x))
    concat_dropped= concat_dropped.withColumn("locationCounty",udf_funCounties("districts"))
    print('locationCounty')
    concat_dropped.show()
    #locationCounty=locCounties(countyData)
    print('locations')
    concat_dropped.show()
    concat_dropped_reset=concat_dropped.drop('id','rokBudowy', 'districts', 'cities_corr', 'region_corr')
    concat_dropped_reset.show()
    #https://stackoverflow.com/questions/35804755/apply-onehotencoder-for-several-categorical-columns-in-sparkmlib
    #https://stackoverflow.com/questions/42805663/e-num-get-dummies-in-pyspark

    rodzajZabudowy = concat_dropped_reset_drop.select("rodzajZabudowy").distinct().rdd.flatMap(lambda x: x).collect()
    materialBudynku = concat_dropped_reset_drop.select("materialBudynku").distinct().rdd.flatMap(lambda x: x).collect()
    stanWykonczenia = concat_dropped_reset_drop.select("stanWykonczenia").distinct().rdd.flatMap(lambda x: x).collect()
    okna = concat_dropped_reset_drop.select("okna").distinct().rdd.flatMap(lambda x: x).collect()
    rynek = concat_dropped_reset_drop.select("rynek").distinct().rdd.flatMap(lambda x: x).collect()


    rodzajZabudowy_expr = [F.when(F.col("rodzajZabudowy") == code, 1).otherwise(0).alias("e_rodzajZabudowy_" + code) for code in rodzajZabudowy]
    materialBudynku_expr = [F.when(F.col("materialBudynku") == code, 1).otherwise(0).alias("e_materialBudynku_" + code) for code in materialBudynku]
    stanWykonczenia_expr = [F.when(F.col("stanWykonczenia") == code, 1).otherwise(0).alias("e_stanWykonczenia_" + code) for code in stanWykonczenia]
    okna_expr = [F.when(F.col("okna") == code, 1).otherwise(0).alias("e_okna_" + code) for code in okna]
    rynek_expr = [F.when(F.col("rynek") == code, 1).otherwise(0).alias("e_rynek_" + code) for code in rynek]



    concat_dummies = concat_dropped_reset_drop.select('rodzajZabudowy',*rodzajZabudowy_expr+materialBudynku_expr+stanWykonczenia_expr+okna_expr+rynek_expr)
    concat_dummies.show()
    #final = pd.get_dummies(concat_dropped_reset_drop, columns=['rodzajZabudowy', 'materialBudynku', 'stanWykonczenia', 'okna', 'rynek'])

    return concat_dummies

In [185]:
filename='/content/drive/MyDrive/domymazowieckie/dfMieszkania1.csv'

In [191]:
final=makeDataFrame(filename)

<class 'pyspark.sql.dataframe.DataFrame'>
powierzchnia_corr
powierzchniaDzialki_corr
cena_corr
rokBudowy_corr
cena/m
lPieter_crr
filtr powierzchnia i cena
districts
cities_corr
region_corr
concat_dropped
+----+------+---------------+---------------+---------+---------------+----------+---------+-----------------+------------------------+--------------+------------------+-----------+-----------+-----------+------------+
|  id|lPokoi| rodzajZabudowy|materialBudynku|rokBudowy|stanWykonczenia|      okna|    rynek|powierzchnia_corr|powierzchniaDzialki_corr|rokBudowy_corr|            cena/m|lPieter_crr|  districts|cities_corr| region_corr|
+----+------+---------------+---------------+---------+---------------+----------+---------+-----------------+------------------------+--------------+------------------+-----------+-----------+-----------+------------+
|null|  null|           blok|           inne|   2012.0|do zamieszkania|plastikowe|   wtórny|             59.7|                    null|    

Py4JJavaError: ignored

In [ ]:
type(final)

In [ ]:
final.limit(10).toPandas()

In [ ]:
final.to_csv('/content/drive/My Drive/domymazowieckie/dfMieszkania_final.csv')

In [ ]:
final.shape

In [ ]:
df=final.copy()

In [ ]:
plt.hist(final.loc[:,'lPokoi']/10)

In [ ]:
sns.boxplot(x=(np.log(final["powierzchnia_corr"])/10))

In [ ]:
sns.boxplot(x=(np.log(final["powierzchniaDzialki_corr"]+1)/14))

In [ ]:
sns.boxplot(x=(np.power(final["rokBudowy_corr"]-1899,4)/3e8))

In [ ]:
plt.hist(np.power(final["rokBudowy_corr"]-1899,4)/3e8)

In [ ]:
sns.boxplot(x=final["cena/m"]/20000)

In [ ]:
sns.boxplot(x=final["lPieter_crr"]/10)

In [ ]:
plt.hist(final["lPieter_crr"]/10)

In [ ]:
final.head()

In [ ]:
plt.hist((final["locationX"]-21)/4)

In [ ]:
plt.hist((final["locationY"]-52)/2)

In [ ]:
final.head()

In [ ]:
#min_max_scaler = preprocessing.MinMaxScaler()
#data_scaled = min_max_scaler.fit_transform(final.values)
#df = pd.DataFrame(data_scaled ,columns=final.columns.values)
#df  = df.fillna(0)

NORMALIZACJA!!!!!!!!!!!!!!!!

In [ ]:
df=final.copy()
df['lPokoi']=final.loc[:,'lPokoi']/30
df["powierzchnia_corr"]=np.log(final["powierzchnia_corr"])/10
df["powierzchniaDzialki_corr"]=np.log(final["powierzchniaDzialki_corr"]+1)/14
df["rokBudowy_corr"]=np.power(final["rokBudowy_corr"]-1899,4)/3e8
df["cena/m"]=final["cena/m"]/20000
df["locationX"]=(final["locationX"]-21)/4
df["locationY"]=(final["locationY"]-52)/2
df['lPieter_crr']=final.loc[:,'lPieter_crr']/10

NORMALIZACJA!!!!!!!!!!!!!!!!!

In [ ]:
df  = df.fillna(0)

In [ ]:
df.head()

In [ ]:
#filename_json1='/content/drive/My Drive/domymazowieckie/dfMazowieckieClean.json'
#df.to_json(filename_json1)

In [ ]:
import tensorflow as tf
import tensorflow as tfCore
import math
from IPython.core.magic import register_line_magic
from IPython.display import Javascript


In [ ]:
train_dataset = df.sample(frac=0.8, random_state=0)
test_dataset = df.drop(df.index)

In [ ]:
train_features = train_dataset.copy()
test_features = test_dataset.copy()

train_labels = train_features.pop('cena/m')
test_labels = test_features.pop('cena/m')

In [ ]:
train_features,test_features,train_labels,test_labels=train_features.values,test_features.values,train_labels.values,test_labels.values

In [ ]:
train_features = np.array(train_features, dtype=np.float32)
test_features = np.array(test_features, dtype=np.float32)
train_labels = np.array(train_labels, dtype=np.float32)
test_labels = np.array(test_labels, dtype=np.float32)

In [ ]:
train_features.shape

In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
import timeit

device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

def cpu():
  with tf.device('/cpu:0'):
    random_image_cpu = tf.random.normal((100, 100, 100, 3))
    net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
    return tf.math.reduce_sum(net_cpu)

def gpu():
  with tf.device('/device:GPU:0'):
    random_image_gpu = tf.random.normal((100, 100, 100, 3))
    net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
    return tf.math.reduce_sum(net_gpu)
  
# We run each op once to warm up; see: https://stackoverflow.com/a/45067900
cpu()
gpu()


In [ ]:
model= tf.keras.models.Sequential([
                               #    normalizer,
  tf.keras.layers.Dense(18*36, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(8*64, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(8*36, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(4*36, activation='relu'),
  tf.keras.layers.Dense(1, activation='linear'),
])

In [ ]:
model.compile(loss='mean_absolute_error',
              metrics=['mse',"mae",'acc'],optimizer=tf.optimizers.Adam(learning_rate=0.01))#learning_rate=0.01

In [ ]:
with tf.device('/device:GPU:0'):
  history = model.fit(
    train_features, train_labels,
    epochs=100,
    # suppress logging
    verbose=0,
    # Calculate validation results on 20% of the training data
    validation_split = 0.2)

In [ ]:
plt.plot(history.history['mse'])
plt.plot(history.history['val_mse'])
plt.ylim(0.01,0.02)

In [ ]:
plt.plot(history.history['mae'])
plt.plot(history.history['val_mae'])
plt.ylim(0.080,0.110)

In [ ]:
zabudowa_list = ["0", "dworek/palac","gospodarstwo","kamienica","szeregowiec","wolnostojacy"]
zabudowa = widgets.Dropdown(options=zabudowa_list, value='0')

material_list = ["0", "pustak","beton komurkowy","cegla","drewno","inne","karmazyt","pustak","silikat"]
material = widgets.Dropdown(options=material_list, value='0')

In [ ]:
zabudowa

In [ ]:
material

In [ ]:
zabudowa_ints=[int(zabudowa.value==element) for element in zabudowa_list]
material_ints=[int(material.value==element) for element in material_list]

In [ ]:
lPokoi = "8" #@param {type:"string"}
powierzchnia = "100" #@param {type:"string"}
powierzchniaDzialki = "100" #@param {type:"string"}
rokBudowy = "100" #@param {type:"string"}
lPieter = "100" #@param {type:"string"}
locationX = "100" #@param {type:"string"}
locationY = "100" #@param {type:"string"}

In [ ]:
lPokoi=4
powierzchnia=60
powierzchniaDzialki=0
rokBudowy=2001
lPieter=5
locationX=21
locationY=52

params_norm=[lPokoi/10,np.log(powierzchnia)/10,np.log(powierzchniaDzialki+1)/14,
             np.power(rokBudowy-1899,4)/3e8,lPieter/30,
             (locationX-21)/4,(locationY-52)/2]

parametry=np.array(params_norm)
zabudowa=np.array([1,0,0,0,0,0,0])
material=np.array([1,0,0,0,0,0,0,0,0])
wykonczenie=np.array([1,0,0,0,0,0])
okna=np.array([1,0,0,0,0])
rynek=np.array([1,0])
x=np.concatenate([parametry,zabudowa,material,wykonczenie,okna,rynek])
x=np.transpose(np.reshape(x,(-1,1)))
print(x)

In [ ]:
cena_m=20000*model.predict(x)[0]
cena=cena_m*powierzchnia
print(cena_m,' ',cena)

In [ ]:
cena_m=20000*model.predict(x)[0]
cena=cena_m*powierzchnia
print(cena_m,' ',cena)

In [ ]:
def szybki_podgląd_lokacji(X,Y):
  lPokoi=7
  powierzchnia=300
  powierzchniaDzialki=1000
  rokBudowy=2001
  lPieter=1
  locationX=X
  locationY=Y

  params_norm=[lPokoi/10,np.log(powierzchnia)/10,np.log(powierzchniaDzialki+1)/14,
              np.power(rokBudowy-1899,4)/3e8,lPieter/10,
              (locationX-21)/4,(locationY-52)/2]

  parametry=np.array(params_norm)
  zabudowa=np.array([0,0,0,0,0,0,1])
  material=np.array([0,0,1,0,0,0,0,0,0])
  wykonczenie=np.array([0,0,0,1,0,0])
  okna=np.array([0,0,0,0,1])
  rynek=np.array([0,1])
  x=np.concatenate([parametry,zabudowa,material,wykonczenie,okna,rynek])
  x=np.transpose(np.reshape(x,(-1,1)))
  cena_m=20000*model.predict(x)[0]
  cena=cena_m*powierzchnia
  return cena_m

In [ ]:
price=[]
for i in range(0,25):
  for j in range(0,55):
   price.append(szybki_podgląd_lokacji(i,j))

In [ ]:
price_1=np.array(price).reshape(i+1,j+1)
price_2=np.transpose(price_1)

In [ ]:
plt.imshow(price_1, cmap='hot')
plt.xlim(40,55)
plt.ylim(18,22)
plt.show()

In [ ]:
filename='/content/drive/My Drive/domymazowieckie/dfMazowieckie.h5'
model.save(filename,save_format='tf')

In [ ]:
pip install tensorflowjs

In [ ]:
!tensorflowjs_converter --input_format=keras /content/drive/MyDrive/domymazowieckie/dfMazowieckie.h5 /content/drive/MyDrive/domymazowieckie/jsmodel